In [6]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

In [7]:
# site : keepjob
home_page="https://www.keejob.com"


In [8]:
joblinks=[]
for i in range(1,5):
    # contact page 
    response=requests.get(f'https://www.keejob.com/offres-emploi/metiers/recrutement-informatique-multimedia/?page={i}')
    sp= BeautifulSoup(response.content,'lxml')
    joblist=sp.find_all("h6")
#     print(joblist)
    for item in joblist:
        for link in item.find_all('a',href=True):
#             print(link)
            joblinks.append(home_page+link['href'])
#             print(joblinks)
# print(joblinks)   
# https://www.keejob.com/offres-emploi/145382/data-scientist-hf/

In [9]:
# testlink="https://www.keejob.com/offres-emploi/145382/data-scientist-hf/"

In [10]:
# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

In [11]:
jobs=[]
for link in joblinks:
    reponse=requests.get(link)
    sp=BeautifulSoup(reponse.content,"lxml")
    job_name=sp.find("h2",class_="job-title").text.strip()
    ps=sp.find("div", class_="text").text.replace('\n', '**').split('**')
    posted_on=ps[6]
    
    
    wk=[sp.find("div", class_="text").text.replace('\t', '').replace('\n','').replace(":","loc: ").strip()]
    place=[]
    for l in wk:
        place.append(re.findall("loc: \W+\w+\W+\w+",l,re.MULTILINE))
        
#     print(place)
#     Workplace= ' '.join(map(str, place))
    for j in place:
        for t in j:
            workplace=t
    
            workplace.replace(" ","")
#     print(workplace)
    ########################################################################
    st=[sp.find("div", class_="text").text.replace('\nBac','Bac').strip()]
    std=[]
    for i in st:
        std.append(re.findall("\w",i,re.MULTILINE))
    
    Study= ' '.join(map(str, std))
    #     print(Study)
    ########################################################################
    
    lg=[sp.find("div", class_="text").text.replace("\t","").replace("Langues:\n\n\n","-").replace("\n,\n\n",",")
       .replace("\n","-")]   
#     print(lg)
    LG=[]
    for i in lg:
        LG.append(re.findall("-[A-Za-z]+.?\w+,\w+-",i,re.MULTILINE))
#     print(LG)
    
#     Languages=st[33:34]+st[35:36]
    
    
    Languages= ' '.join(map(str, LG))
#     print(Languages)
    
    #################################################################################
    cmp=sp.find("div", class_="span9 content").text.replace('\n', '**').strip().split('**')
    # print(cmp)
    Company_name=cmp[2]
    Company_name= ' '.join(map(str, Company_name))
    # print(Company_name)
    Job_description=sp.find("div", class_="block_a span12 no-margin-left").text.replace('Imprimer','').replace('\t','').replace('«\xa0','').replace('-\xa0',' ').replace('\xa0','').replace('\n',' ').replace('\r',' ').split('**')
    Job_description = ' '.join(map(str, Job_description))
    sentence = nltk.sent_tokenize(Job_description)

    print(sentence)
    # link_img=[]
    # img_cmp=sp.find_all('img')
    # link_img=[]
    # for image in img_cmp:

    #     link_img=image['src']
    # #     print(link_img)

    # #     print(link_img)
    # print(link_img)            



#     # print(name,"\n",posted_on,"\n",Workplace,"\n",Study,"\n",Languages,"\n",Company_name,"\n",Job_description)
    job={
         'job_name':job_name,
         'posted_on':posted_on,
         'workplace':workplace,
        'Study':Study,
         'Languages':Languages,
         'Company_name':Company_name,
         'Job_description':Job_description
    }
#     print(job)
 
    jobs.append(job)
#     print('saving job: ',jobs)







[" Description de l'annonce: Agence Télécom Services, Filiale du Premier distributeur indépendant deSFR BusinessBasé en France, cherche à recruter un(e)Technicien(ne) Réseaux et Système pour sa filiale basée à Tunis.Vos missions:  Analyser et résoudre les incidents déclarés- Gérer la mise en service des routeurs et switch- Prendre en charge des demandes et accompagner les techniciens terrain- Administrer le systèmeVos Compétences:DHCP, DNS, AD, LAN, WAN, VLAN, TCP/IP, VPN, WIFI, HSRP, Routage, Cisco, Fortinet, VRF, VRRP, Windows,Architecture informatique Savoir analyser les besoins- Paramétrage de switch, routeur,- Commande Cisco- Suivi supervision,- connaissance lien ADSL, SDSL,- Connaissance des réseaux operateurs et des protocoles télécom (BGP, MPLS…)- connaissance des dysfonctionnements réseau (test d'état du câblage, des équipements)- Traitement des demandes en respectant les procédures.- Réalisation d'un diagnostic précis- Recherche de la solution technique appropriée.- Réponse t

[" Description de l'annonce: Industrie  pharmaceutique de renommée basée à Sousse recrute deux développeurs Web  Full-Stack titulaire d'un diplôme d'ingénieur / Master/ Bac + 5, ayant des  bonnes connaissances en développement Web:Langage : CSS, HTML5,JavaScript Framework :Angular JS, React JS, Type script, Jquery, Redux Outils: Node.js, Git, Gitlab, Visual    Studio SGBD: MS SQL , PostgreSQL, Sql Serveur Les outils CI/CD et Architectures modernes.Méthodologies Agile (scrum)       Rattaché au département informatique, il aura pour missions de:  Participer à la phase de    conception.", 'Analyser les projets en    fonction de entretiens avec les utilisateurs et les responsables des    activités.', 'Participer à la définition des URS en concertation avec les    parties Prenantes  Elaboration des cahiers des charges techniques et    fonctionnelles.', 'Développer des applications performantes pour digitaliser    les processus et augmentant la productivité dans les différents départements  

[" Description de l'annonce: Nous recrutons pour le compte d'une multinationale spécialisée dans la production de produits électroniques,Chargé  d’Exploitation ERP (Support Informatique)MISSIONS:    Créer et suivre les droits d'accès, comptes       utilisateurs en fonction des profils Qualifier les incidents entrants et établir un       premier diagnostic pour intervenir ou escalader suivant le niveau de       l'incident Résoudre les incidents en fonction des consignes et procédures       en vigueur Apporter le support et la formation nécessaires       aux utilisateurs, les sensibiliser à la sécurité et aux bonnes pratiques Être garant de la mise à jour des documentations       : suivis d'incidents, comptes rendus, optimisation de procédures Être force de proposition sur l'automatisation       des diverses tâches récurrentes et l'amélioration des processus internes Diagnostiquer la nature et l'origine des       incidents, les transmettre au niveau 2 du support Veiller à la cohérence de

[" Description de l'annonce: Vous souhaitez intégrer une entreprise dynamique et innovante en forte croissance ?", 'Rejoignez-nous!Leader européen des assurances et services affinitaires pour de grandes marques, SPB, courtier gestionnaire compte près de 1800 collaborateurs répartis dans 17 pays.', 'Entreprise familiale indépendante fondée en 1965, SPB opère en marque blanche (B2BtoC).', 'Notre métier : accompagner la conception, le développement et gérer des programmes d’assurances, d’extensions de garantie et de services associés à des produits de téléphonie mobile, nomades, bancaires ou encore liés à l’énergie, la santé, le voyage, les loisirs, la prévoyance ainsi qu’aux biens de la maison.Tâches principales:* Réaliser les tâches affectées dans le respects des charges et délais convenus.', "* Participer à l'analyse et à la conception technique.", '* Faire la maintenance évolutive, corrective et palliative.', '* Améliorer et optimiser les solutions existantes.', "* Respecter les règle

[" Description de l'annonce: Binit  Nearshore Services (BinitNS) est une société de Conseil et de Services opérant  dans le domaine de l’ITO et du BPO.", 'Nous conseillons et nous accompagnons nos  clients Européens dans le cadre de l’installation en Tunisie de leurs équipes  nearshore en extension de leurs équipes européennes.', 'Un  nouveau client monte chez nous une équipe de développement et une équipe QA en  prolongement de ses équipes en Europe.', 'Dans ce  cadre, nous cherchons à recruter un développeur BI, capable d’effectuer la conception, le développement et la maintenance de  dashboards et d’éléments de reporting sous l’outil Power BI, ainsi que la réalisation  des tests.', 'L’environnement technologique est très intéressant.', 'Vous  allez travailler principalement sur BI Tools, power BI , SQLServer et toutes  les technologies de gestion des données dans l’environnement Microsoft.', 'Si un tel challenge vous intéresse et  que vous avez fait de bons stages ou avez 1 à 2 ans 

[" Description de l'annonce: Rattaché au  Directeur Commercial.", "Vous  fournissez un  leadership éclairé sur la façon dont les scénarios et les fonctionnalités Cloud  seront mis en œuvre dans l'entreprise, capable de présenter et de représenter  les services Cloud proposés.", "Vous  collaborez avec l’équipe technique Cloud pour s'assurer que les livrables sont  définis correctement avec des critères d'acceptation appropriés et s'assurer  que les produits finaux acceptés sont de la plus haute qualité.", "Vous  êtes responsable de la collecte et de la définition des cas d'utilisation commerciale,  de la définition des critères d'acceptation.", "Vous  gérez et publier des feuilles de route pour s'assurer que les parties prenantes  sont informées, vous donnez les inputs nécessaires pour élaborer le Business  Plan de la solution proposée au client permettant d’apprécier la rentabilité de  l’offrefinancière.", 'Vous  avez un diplôme d’ingénieur spécialisé en informatique ou en  télécommuni

[" Description de l'annonce: En 2006, lorsque NOZ (le leader européen du déstockage) a commencé son expansion en Europe de l'Est et en Afrique, Veo est très vite tombé amoureux de sa vision et de son environnement agile, cherchant toujours à découvrir davantage sur cette extraordinaire activité qui est le déstockage et l'économie responsable.Aujourd'hui, Veo Worldwide est un fournisseur de services B2B, implanté en Europe, qui compte 15 ans d'activité.", 'Nous recherchons constamment des solutions pour aider les propriétaires d\'entreprises à capitaliser sur leurs stocks invendus (fins de séries, produits à date courte, commandes annulées) et à devenir efficaces.Grâce à tout cela, Veo consolide son renom basé surl’engagement, la transparence, la solidarité, l\'amélioration continue et la compétencePour renforcer notre équipe à Tunis, nous souhaitons recruter "Team Leader Web, Mobile"Principales responsabilités:- Manager et animer une équipe de 5 à 7 personnes- Gestion des projets- Assu

[" Description de l'annonce: A propos de nous: Timsoft Group est un one-stop-shop de la transformation digitale des  entreprises.", 'Fort de son expérience de leader des solutions de gestion, Timsoft  est un éditeur, distributeur et intégrateur de solutions de gestion pour les  professionnels des métiers du retail et des industries textiles et  manufacturières.', 'Fondé en 2003, Timsoft a élargi ses opérations au-delà des frontières  tunisiennes avec l’ouverture de bureaux en France et au Maroc, offrant ainsi  ses services dans plus de 30 pays africains, européens et du moyen orient.', 'Timsoft, c’est aussi +120 collaborateurs dont +70 consultants et +1400 projets  délivrés.', 'Nous avons l’ambition d’accompagner les grands Retailers sur  leurs projets de transformation digitale et nous sommes à la recherche de  potentiels, de talents pour renforcer nos équipes.', 'Ce poste vous offre la possibilité de travailler dans une équipe  dynamique de consultants et de mettre à profit vos conna

[" Description de l'annonce: Missions   – Installation et configuration système – Installation et configuration des réseaux  informatique -Installer, configurer et maintenir les serveurs : ·  Installation  et configuration des serveurs/Machines Linux, Windows ·  Administration  des plateformes de virtualisation ·  Configurer,  administrer les services d’infrastructure : services DNS, DHCP, RADIUS, PROXIES  …etc.", '·  Diagnostiquer  et résoudre les dysfonctionnements – Résoudre toutes les pannes dans le domaine informatique – Monter, installer et mettre en service les  nouveaux matériels informatiques  – Diagnostiquer les défaillances et proposer des  solutions de réparations  – Organiser la surface de travail Profil: De formation bac+3 en informatique, vous justifiez  impérativement d’une expérience significative d’au moins 3 ans dans une  fonction similaire en entreprise.', 'Les savoir-faire Connaissance approfondie de l’architecture  matérielle des serveurs et des postes de travail 

[" Description de l'annonce:  Ce que vous allez faire Vous allez rejoindre une équipe travaillant en mode DevSecOps.", 'Cela signifie que notre rôle est à la fois de créer l’application (Dev), en intégrant les problématiques de sécurité (Sec) mais aussi d’assurer sa stabilité, ses performances et son bon fonctionnement (Ops).', 'Votre rôle dans l’équipe sera d’être un expert infrastructure et des services AWS.', "Pour les applications dont nous récupérons uniquement la gestion de l’infrastructure: ● Migrer les infras existantes vers AWS ; ● Les faire évoluer pour qu’elles utilisent les services managés AWS ; ● Mettre en place les outils et procédures d'exploitation ; Pour les applications que nous gérons totalement, en collaboration avec les développeurs : ● Définir l’infrastructure optimale pour l’application ; ● Automatiser la création de l’infra et les déploiements ; ● Assurer la stabilité et la performance de l’application ; Vos points forts Vous êtes curieux, vous aimez découvrir 

In [25]:
# formatted_article_text = re.sub('[^a-zA-Z]', ' ', sentence )
# formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)
# stopwords = nltk.corpus.stopwords.words('french')

# word_frequencies = {}
# for word in nltk.word_tokenize(formatted_article_text):
#     if word not in stopwords:
#         if word not in word_frequencies.keys():
#             word_frequencies[word] = 1
#         else:
#             word_frequencies[word] += 1


TypeError: expected string or bytes-like object

In [13]:
df = pd.DataFrame(jobs )
df

,Company_name,Job_description,Languages,Study,job_name,posted_on,workplace
0,A T S E R V I C E S,Description de l'annonce: Agence Télécom Serv...,[],"['D', 'é', 't', 'a', 'i', 'l', 's', 'd', 'e', ...",Technicien Réseaux et Système,6 février 2022,loc: La Marsa
1,C A L L M A X,Description de l'annonce: CallMaxest une ent...,"['-Arabe,Français-']","['D', 'é', 't', 'a', 'i', 'l', 's', 'd', 'e', ...",Technicien Helpdesk,6 février 2022,"loc: Tunis, ..."
2,C O N C E N T R I X,Description de l'annonce: Profil recherché :B...,"['-Français,Anglais-']","['D', 'é', 't', 'a', 'i', 'l', 's', 'd', 'e', ...",Conseiller Client réceptions d'appels,6 février 2022,"loc: Ariana, ..."
3,S P B I S C,Description de l'annonce: Vous souhaitez int...,[],"['D', 'é', 't', 'a', 'i', 'l', 's', 'd', 'e', ...",Business Analyst,6 février 2022,"loc: Tunis, ..."
4,P H O N E A C T,"Description de l'annonce: PhoneAct,spécialist...",[],"['D', 'é', 't', 'a', 'i', 'l', 's', 'd', 'e', ...",Administrateur Système et réseaux,6 février 2022,"loc: Ariana, ..."
5,,Description de l'annonce: Industrie pharmace...,"['-Français,Anglais-']","['D', 'é', 't', 'a', 'i', 'l', 's', 'd', 'e', ...",Développeurs Web Full-Stack,6 février 2022,loc: Kalaa El
6,,Description de l'annonce: Vous Assistez une ...,"['-Français,Anglais-']","['D', 'é', 't', 'a', 'i', 'l', 's', 'd', 'e', ...",LEAD DEV/ Chargé de pilotage,6 février 2022,loc: Ariana Ville
7,S T R E A M W I D E,Description de l'annonce: As part of the Mark...,"['-Arabe,Français,Anglais-']","['D', 'é', 't', 'a', 'i', 'l', 's', 'd', 'e', ...",Product Marketing Manager,6 février 2022,"loc: Tunis, ..."
8,,Description de l'annonce: Un laboratoire phar...,"['-Arabe,Français,Anglais-']","['D', 'é', 't', 'a', 'i', 'l', 's', 'd', 'e', ...",Administrateur Système SAGE X3,6 février 2022,loc: Ben Arous
9,A D P E S T U N I S I E,Description de l'annonce: Le Groupe ADP (N°1 ...,"['-Arabe,Français,Anglais-']","['D', 'é', 't', 'a', 'i', 'l', 's', 'd', 'e', ...",Développeur COBOL,6 février 2022,loc: Bab Bhar


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse


In [20]:
X = df['Job_description']
y = df['job_name']

VecModel = TfidfVectorizer()
X_Vec = VecModel.fit_transform(X)

feature_names = VecModel.get_feature_names()
feature_names


['009158',
 '10',
 '100',
 '1119',
 '120',
 '13',
 '130',
 '1400',
 '15',
 '1500',
 '168m',
 '17',
 '180',
 '1800',
 '1965',
 '1pl',
 '20',
 '200',
 '2000',
 '2003',
 '2006',
 '2008',
 '2008r2',
 '2012',
 '2014',
 '2016',
 '2020',
 '2022',
 '24',
 '27001',
 '28',
 '2pl',
 '30',
 '300',
 '365',
 '37',
 '3pl',
 '40h',
 '4g',
 '4pl',
 '5g',
 '5mois',
 '70',
 '850',
 '99',
 'abab',
 'abap',
 'ability',
 'able',
 'aboutissent',
 'abstenir',
 'academy',
 'acceptance',
 'acceptation',
 'accepter',
 'acceptés',
 'accessoires',
 'accompagnant',
 'accompagne',
 'accompagnement',
 'accompagner',
 'accompagnons',
 'according',
 'account',
 'accueilli',
 'accès',
 'achat',
 'achiever',
 'acknowledge',
 'acquisition',
 'across',
 'act',
 'acteur',
 'actions',
 'active',
 'activement',
 'activities',
 'activité',
 'activités',
 'actualité',
 'actuellement',
 'ad',
 'adaptabilité',
 'adaptation',
 'adapter',
 'adaptez',
 'adaptive',
 'adaptées',
 'added',
 'administrateur',
 'administrateurs',
 'admin

In [24]:

# c=pd.Series(X_Vec)
# c.plot(kind='pie')

In [22]:
# X_Vec = pd.DataFrame(X_Vec)
# X_Vec = pd.DataFrame.spars.from_spmatrix(X_Vec)

X_Vec = pd.SparseDataFrame(X_Vec)
print(f'The new shape for X is {X_Vec.shape}')
X_Vec.head()


The new shape for X is (60, 2769)


,0,1,2,3,4,5,6,7,8,9,...,2759,2760,2761,2762,2763,2764,2765,2766,2767,2768
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.042664,NaN,NaN
1,NaN,NaN,0.077174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.042710,0.054016,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.118502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,0.050543,NaN,NaN,NaN,NaN,NaN,NaN,0.055651,...,NaN,0.044108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.034176
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
for i in range(5) : 
    print(sorted(list(X_Vec.iloc[i,:]),reverse = True))
    print('--------------------------------------------------')

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.05799209815681837, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0845122741369244, nan, nan, nan, nan, nan, nan, nan, nan, 0.06349553191642741, nan, nan, nan, nan, nan, nan, 0.0845122741369244, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0845122741369244, nan, nan, nan, nan, nan, nan, 0.0845122741369244, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.04870420586253956, nan, 0.11147775537196684, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 

[nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.05123335694863036, nan, nan, nan, 0.05123335694863036, nan, nan, nan, nan, nan, 0.045459624178754196, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.05123335694863036, nan, nan, nan, nan, nan, nan, 0.05123335694863036, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.09814359229589677, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.05123335694863036, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.04391677538101654, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.08242512126984498, nan, 0.09433032879499008, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.06539268880

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_Vec, y, test_size=0.25, random_state=42)